In [8]:
import cv2
import numpy as np
import os
import glob

# Ruta de la carpeta con las imágenes
folder_path = r"C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks"  # Cambia esta ruta a tu carpeta de imágenes

# Listar todas las imágenes en la carpeta
image_files = glob.glob(os.path.join(folder_path, "*.png"))  # Cambia el formato si es necesario

# Definir la altura deseada (250px)
target_height = 250
target_width = 250

for image_path in image_files:
    # Leer la imagen en escala de grises
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Añadir un borde negro de 200 píxeles alrededor de la imagen
    img_with_border = cv2.copyMakeBorder(img, 200, 200, 200, 200, cv2.BORDER_CONSTANT, value=0)
    
    # Umbralizar la imagen para obtener una imagen binaria
    _, thresh = cv2.threshold(img_with_border, 127, 255, cv2.THRESH_BINARY)
    
    # Encontrar los contornos
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Crear una máscara binaria vacía para dibujar los contornos rotados
    mask = np.zeros_like(img_with_border)
    
    for contour in contours:
        # Ajustar la elipse al contorno
        if len(contour) >= 5:  # Necesitamos al menos 5 puntos para fitEllipse
            (x, y), (MA, ma), angle = cv2.fitEllipse(contour)
            
            # Calcular el ángulo de rotación
            rotation_angle = 180 - angle
            M = cv2.moments(contour)
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            
            # Normalizar los puntos del contorno
            cnt_norm = contour - [cx, cy]
            
            coordinates = cnt_norm[:, 0, :]
            xs, ys = coordinates[:, 0], coordinates[:, 1]
            
            # Convertir de coordenadas cartesianas a polares
            thetas = np.arctan2(ys, xs)
            rhos = np.sqrt(xs**2 + ys**2)
            
            # Convertir los ángulos de radianes a grados y realizar la rotación
            thetas_deg = np.rad2deg(thetas)
            thetas_new_deg = (thetas_deg + rotation_angle) % 360
            thetas_new = np.deg2rad(thetas_new_deg)
            
            # Convertir de nuevo las coordenadas polares a cartesianas
            xs_rot, ys_rot = rhos * np.cos(thetas_new), rhos * np.sin(thetas_new)
            
            # Actualizar el contorno con las coordenadas rotadas
            cnt_norm[:, 0, 0] = xs_rot
            cnt_norm[:, 0, 1] = ys_rot
            cnt_rotated = cnt_norm + [cx, cy]
            cnt_rotated = cnt_rotated.astype(np.int32)
            
            # Dibujar el contorno rotado en la máscara binaria
            cv2.drawContours(mask, [cnt_rotated], -1, (255), thickness=cv2.FILLED)
    
    # Encontrar la caja delimitadora (bounding box) del contorno
    x, y, w, h = cv2.boundingRect(cnt_rotated)
    
    # Recortar la región de interés (ROI) desde la máscara
    ROI = mask[y:y + h, x:x + w]
    
    # Escalar la imagen manteniendo la altura constante
    factor_escala = target_height / ROI.shape[0]
    nueva_h = target_height
    nueva_w = int(ROI.shape[1] * factor_escala)
    
    # Redimensionar la ROI
    ROI_redimensionada = cv2.resize(ROI, (nueva_w, nueva_h), interpolation=cv2.INTER_NEAREST)
    
    # Si el ancho es menor que 250px, añadimos bordes negros
    if nueva_w < target_width:
        delta_w = target_width - nueva_w
        left_padding = delta_w // 2
        right_padding = delta_w - left_padding
        ROI_redimensionada = cv2.copyMakeBorder(ROI_redimensionada, 0, 0, left_padding, right_padding, cv2.BORDER_CONSTANT, value=0)
    
    # Asegurarse que la imagen final tiene 250x250 px
    final_image = cv2.resize(ROI_redimensionada, (target_width, target_height), interpolation=cv2.INTER_NEAREST)

    # Guardar o mostrar la imagen procesada
    output_path = os.path.join(folder_path, "resized_" + os.path.basename(image_path))
    cv2.imwrite(output_path, final_image)
    print(f"Procesado: {output_path}")

Procesado: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_1.png
Procesado: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_10.png
Procesado: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_11.png
Procesado: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_12.png
Procesado: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_2.png
Procesado: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_3.png
Procesado: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_4.png
Procesado: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_5.png
Procesado: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_6.png
Procesado: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almon

In [2]:
import cv2
import os

# Lista de rutas de imágenes
rutas_imagenes = [
    r'C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_5.png',
    r'C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_6.png',
    r'C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_7.png',
      r'C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_8.png',
        r'C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_10.png',
          r'C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_11.png', 
           r'C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\resized_Almond_type_12.png', # Agrega más rutas según sea necesario
]

# Carpeta de salida para las imágenes giradas
carpeta_salida = 'imagenes_rotadas'
os.makedirs(carpeta_salida, exist_ok=True)  # Crear carpeta si no existe

for ruta in rutas_imagenes:
    # Cargar la imagen
    imagen = cv2.imread(ruta)
    
    if imagen is None:
        print(f"Error: No se pudo cargar la imagen en {ruta}")
        continue

    # Girar la imagen 180°
    imagen_girada = cv2.rotate(imagen, cv2.ROTATE_180)

    # Obtener el nombre de la imagen original
    nombre_imagen = os.path.basename(ruta)

    # Crear la ruta de salida
    ruta_salida = os.path.join(carpeta_salida, f"{nombre_imagen}")

    # Guardar la imagen girada
    cv2.imwrite(ruta_salida, imagen_girada)
    print(f"Imagen guardada en: {ruta_salida}")

print("Proceso completado.")

Imagen guardada en: imagenes_rotadas\resized_Almond_type_5.png
Imagen guardada en: imagenes_rotadas\resized_Almond_type_6.png
Imagen guardada en: imagenes_rotadas\resized_Almond_type_7.png
Imagen guardada en: imagenes_rotadas\resized_Almond_type_8.png
Imagen guardada en: imagenes_rotadas\resized_Almond_type_10.png
Imagen guardada en: imagenes_rotadas\resized_Almond_type_11.png
Imagen guardada en: imagenes_rotadas\resized_Almond_type_12.png
Proceso completado.


In [1]:
import cv2
import numpy as np
import os
import glob

# Ruta de la carpeta con las imágenes
folder_path = r"C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\BACKUP"  # Cambia esta ruta a tu carpeta de imágenes

# Listar todas las imágenes en la carpeta
image_files = glob.glob(os.path.join(folder_path, "*.png"))  # Cambia el formato si es necesario

# Definir la altura deseada (250px)
target_height = 250
target_width = 250

for image_path in image_files:
    # Leer la imagen en escala de grises
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        # Invertir los colores de la imagen
        inverted_img = cv2.bitwise_not(img)
        
        # Sobrescribir la imagen original con la imagen invertida
        cv2.imwrite(image_path, inverted_img)
        print(f"Imagen procesada y guardada: {image_path}")
    else:
        print(f"No se pudo leer la imagen: {image_path}")

Imagen procesada y guardada: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\BACKUP\resized_Almond_type_1.png
Imagen procesada y guardada: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\BACKUP\resized_Almond_type_10.png
Imagen procesada y guardada: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\BACKUP\resized_Almond_type_11.png
Imagen procesada y guardada: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\BACKUP\resized_Almond_type_12.png
Imagen procesada y guardada: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\BACKUP\resized_Almond_type_2.png
Imagen procesada y guardada: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\BACKUP\resized_Almond_type_3.png
Imagen procesada y guardada: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\BACKUP\resized_Almond_type_4.png
Imagen procesada y guardada: C:\Users\Pheno\Documents\database_almondcv2\MORPHOMETRICS\masks\BACKUP\resized_Almond_